## PhidgetFour_v1 notebook

November 19. M. Lamoureux

Test the four plant Phidgets attached to the VINT device.

#### Setup

- plug in the Phidget VINT Hub to the computer's USB port
- Plug in the temperature/humidity, moisture, and light sensor to the VINT
- The moisture probe must be on Port 1, the other sensors can be anywhere.
- (You can use a different port, but you need to change the code below.)
- The code below creates a phidgetFour object in Python, sets the port number to 1, then displays the phidgetFour widget
- click on the button on the screen to connect the phidget and all four sensors
- you can see the values displayed. Blow on the sensors to change values
- additional code links the physical sensors with Plotly gauges on screen.



## Pip install

The pip install only needs to be done once on your hub. I include it here for completeness.

In [ ]:
!pip install "anywidget[dev]"

In [2]:
import anywidget
import traitlets

In [8]:
class PhidgetsFour(anywidget.AnyWidget):
    _esm = """
import  "https://unpkg.com/phidget22/browser/phidget22";

export function render({ model, el }) {
    var conn = 0;
    var chTempe = 0;
    var chHumid = 0;
    var chMoist = 0;
    var chLumin = 0;
    var isOpen = false;
    var connOpen = false;
    var chOpen = false;

    let getTemperature = () => model.get("temperature");
    let getHumidity = () => model.get("humidity");
    let getMoisture = () => model.get("moisture");
    let getLuminance = () => model.get("luminance");

    async function openUSB() {
        if (conn) {
            try {await chTempe.close();} catch {console.log('chT.close failed');}
            try {await chHumid.close();} catch {console.log('chH.close failed');}
            try {await chMoist.close();} catch {console.log('chM.close failed');}
            try {await chLumin.close();} catch {console.log('chL.close failed');}
            try {await conn.close();} catch {console.log('conn.close failed');}
            try {await conn.delete();} catch {console.log('conn.delete failed');}
        }
        conn = new phidget22.USBConnection();
        chTempe = new phidget22.TemperatureSensor();
        chHumid = new phidget22.HumiditySensor();
        chMoist = new phidget22.VoltageRatioInput();
        chLumin = new phidget22.LightSensor();
        conn.onError = function(code, msg) {
            console.log('Conn Code: ' + code);
            console.log('Conn error: ' + msg);
        };
        chTempe.onError = (code, msg) => {
        	console.log('Tempe Code: ' + code);
        	console.log('Tempe error: ' + msg);
        };
        chHumid.onError = (code, msg) => {
        	console.log('Humid Code: ' + code);
        	console.log('Humid error: ' + msg);
        };
        chMoist.onError = (code, msg) => {
        	console.log('Moist Code: ' + code);
        	console.log('Moist error: ' + msg);
        };
        chLumin.onError = (code, msg) => {
        	console.log('Lumin Code: ' + code);
        	console.log('Lumin error: ' + msg);
        };
        chTempe.onTemperatureChange = function(temperature) {
            textTempe.innerHTML = 'Temperature is ' + temperature;
            model.set('temperature', temperature);
            model.save_changes();
       	};
        chHumid.onHumidityChange = function(humidity) {
            textHumid.innerHTML = 'Humidity is ' + humidity;
            model.set('humidity', humidity);
            model.save_changes();
       	};
        chMoist.onVoltageRatioChange = function(ratio) {
            textMoist.innerHTML = 'Moisture is ' + ratio;
            model.set('moisture', ratio);
            model.save_changes();
       	};
        chLumin.onIlluminanceChange = function(luminance) {
            textLumin.innerHTML = 'Luminance is ' + luminance;
            model.set('luminance', luminance);
            model.save_changes();
       	};
        chTempe.onAttach = onAttach;
        chHumid.onAttach = onAttach;
        chMoist.onAttach = onAttach;
        chLumin.onAttach = onAttach;
        chTempe.onDetach = () => {
        	console.log('T onDetach called. ');
        }
        chHumid.onDetach = () => {
        	console.log('H onDetach called. ');
        }
        chMoist.onDetach = () => {
        	console.log('M onDetach called. ');
        }
        chLumin.onDetach = () => {
        	console.log('L onDetach called. ');
        }

        chMoist.setIsHubPortDevice(true);
        chMoist.setHubPort(model.get("hubPort"));
    	try {
    		await conn.connect();
            conn.requestWebUSBDeviceAccess();
            console.log('usb connected');
            connOpen = true;
    	} catch(err) {
    		console.log("Conn open error: ", err);
            connOpen = false;
    	}
        if (connOpen) {
            try {
                await chTempe.open();
                await chHumid.open();
                await chMoist.open();
                await chLumin.open();
                console.log('Channel connected');
                chOpen = true;            
            } catch(err) {
        		console.log("Channel open error: ", err);
                chOpen = false;
            }
        }
        // now get the UI to reflect the status
        if (connOpen && chOpen) {
            isOpen = true;
            textStatus.innerHTML = 'USB connected, channels open.';
            button.innerHTML = `Click to disconnect`;
        }
        if (connOpen && !chOpen) {
            isOpen = false;
            textStatus.innerHTML = 'USB connected, channels not open.';
            button.innerHTML = `Click to connect`;
        }
        if (!connOpen) {
            isOpen = false;
            textStatus.innerHTML = 'USB did not connect.';
            button.innerHTML = `Click to connect`;        
        }
    };
    function onAttach(ch) {
        console.log('channel is attached' + ch);
        textStatus.innerHTML = 'Attached.  ';
        ch.setDataInterval(1000);
    }
    async function closeUSB() {
        console.log('closeUSB called');
        try {await chTempe.close();} catch {console.log('T.close failed');}
        try {await chHumid.close();} catch {console.log('H.close failed');}
        try {await chMoist.close();} catch {console.log('M.close failed');}
        try {await chLumin.close();} catch {console.log('L.close failed');}
        try {await conn.close();} catch {console.log('conn.close failed');}
        try {await conn.delete();} catch {console.log('conn.delete failed');}
        isOpen = false; 
        connOpen = false;
        chOpen = false;
        textStatus.innerHTML = 'Disconnected.  ';
        textTempe.innerHTML = 'Temperature is null.';
        textHumid.innerHTML = 'Humidity is null.';
        textMoist.innerHTML = 'Moisture is null.';
        textLumin.innerHTML = 'Luminance is null.';
        button.innerHTML = `Click to connect`;
    };
    model.on('msg:custom', msg => {
        console.log(`new message: ${JSON.stringify(msg)}`);
        if (msg == 'open') {
            console.log('open called');
            openUSB();
        }
        if (msg == 'close') {
            console.log('close called');
            closeUSB();
        }
    });

    let button = document.createElement("button");
    button.classList.add("ph-button");
    button.innerHTML = `Click to open USB`;
    button.addEventListener("click", async () => {
        if (isOpen) {closeUSB();} else {openUSB();}
    });
    let textStatus = document.createElement("label");
    textStatus.innerHTML = 'Status message here';
    let textTempe = document.createElement("label");
    textTempe.innerHTML = 'Temperature';
    let textHumid = document.createElement("label");
    textHumid.innerHTML = 'Humidity';
    let textMoist = document.createElement("label");
    textMoist.innerHTML = 'Moisture';
    let textLumin = document.createElement("label");
    textLumin.innerHTML = 'Luminance';
    model.on("change:temperature", () => {
        textTempe.innerHTML = `Temperature is ${getTemperature()}`;
    });
    model.on("change:humidity", () => {
        textHumid.innerHTML = `Humidity is ${getHumidity()}`;
    });
    model.on("change:moisture", () => {
        textMoist.innerHTML = `Moisture is ${getMoisture()}`;
    });
    model.on("change:luminance", () => {
        textLumin.innerHTML = `Luminance is ${getLuminance()}`;
    });
    let div0 = document.createElement("div");
    let div1 = document.createElement("div");
    let div2 = document.createElement("div");
    let div3 = document.createElement("div");
    div0.appendChild(textTempe);
    div1.appendChild(textHumid);
    div2.appendChild(textMoist);
    div3.appendChild(textLumin);

    el.appendChild(button);
    el.appendChild(textStatus);
    el.appendChild(div0);
    el.appendChild(div1);
    el.appendChild(div2);
    el.appendChild(div3);
    // we include a return function to close the Phidget when the notebook is close
    return async () => {
        try {await chTempe.close();} catch {}
        try {await chHumid.close();} catch {}
        try {await chMoist.close();} catch {}
        try {await chLumin.close();} catch {}
        try {await conn.close();} catch {}
        try {await conn.delete();} catch {}
    };
    }
    """
    _css="""
    .ph-button {color: white; 
                background-color: var(--md-blue-700); 
                border-radius: 8px; 
                font-size: 24px; 
                display: block;
                padding: 15px 32px;}
    .ph-button:hover { background-color: var(--md-blue-900); }
    """
    hubPort = traitlets.Int(0).tag(sync=True)   
    temperature = traitlets.Float(0).tag(sync=True)
    humidity = traitlets.Float(0).tag(sync=True)
    moisture = traitlets.Float(0).tag(sync=True)
    luminance = traitlets.Float(0).tag(sync=True)

pf = PhidgetsFour()
pf.hubPort = 1


In [9]:
pf

PhidgetsFour(hubPort=1)

In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets

In [11]:
# the four sensor gauges
g_temp = go.Indicator(
    mode = "gauge+number",
    value = 20,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Temperature"},
    gauge = {'axis': {'range': [10, 40]}}
)

g_hum = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Humidity"},
    gauge = {'axis': {'range': [0, 100]}}
)

g_moist = go.Indicator(
    mode = "gauge+number",
    value = 0.5,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Moisture"},
    gauge = {'axis': {'range': [0, 1.0]}}
)

g_light = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Light Level"},
    gauge = {'axis': {'range': [0, 10000]}}
)

fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type' : 'domain'}, {'type' : 'domain'}],[{'type' : 'domain'}, {'type' : 'domain'}]],
    vertical_spacing = 0.35
)
fig.append_trace(g_temp, row=1, col=1)
fig.append_trace(g_hum, row=1, col=2)
fig.append_trace(g_moist, row=2, col=1)
fig.append_trace(g_light, row=2, col=2)

gauges = go.FigureWidget(fig)

In [12]:
def updateTemperature(change):
    gauges.data[0]['value'] = change.new
    
def updateHumidity(change):
    gauges.data[1]['value'] = change.new
    
def updateMoisture(change):
    gauges.data[2]['value'] = change.new
    
def updateLuminance(change):
    gauges.data[3]['value'] = change.new


In [13]:
pf.observe(updateTemperature, names=["temperature"])
pf.observe(updateHumidity, names=["humidity"])
pf.observe(updateMoisture, names=["moisture"])
pf.observe(updateLuminance, names=["luminance"])

In [14]:
gauges

FigureWidget({
    'data': [{'domain': {'x': [0.0, 0.45], 'y': [0.675, 1.0]},
              'gauge': {'axis': {'range': [10, 40]}},
              'mode': 'gauge+number',
              'title': {'text': 'Temperature'},
              'type': 'indicator',
              'uid': 'b82ce543-21c1-4473-b160-8a7e3012ee60',
              'value': 20},
             {'domain': {'x': [0.55, 1.0], 'y': [0.675, 1.0]},
              'gauge': {'axis': {'range': [0, 100]}},
              'mode': 'gauge+number',
              'title': {'text': 'Humidity'},
              'type': 'indicator',
              'uid': '7bea46a5-5052-40a2-abd4-284d2b298cd1',
              'value': 40},
             {'domain': {'x': [0.0, 0.45], 'y': [0.0, 0.325]},
              'gauge': {'axis': {'range': [0, 1.0]}},
              'mode': 'gauge+number',
              'title': {'text': 'Moisture'},
              'type': 'indicator',
              'uid': 'd84521df-43cd-4b18-96a8-58fc9562b1b0',
              'value': 0.5},
         